In [1]:
import sqlite3
import pandas as pd
import json

In [2]:
def get_tables(path=None,con=None):
    if path:
        con=sqlite3.connect(path)
    cur=con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    res= cur.fetchall()
    con.close
    res_data=[r[0] for r in res]
    return res_data
get_tables("pack01")

['test_category', 'test', 'test_question', 'question', 'answer']

In [3]:
def get_metadata(path=None,con=None):
    if path:
        con=sqlite3.connect(path)
    cur=con.cursor()
    sql="""
        SELECT
      m.name AS table_name, 
      p.cid AS col_id,
      p.name AS col_name,
      p.type AS col_type,
      p.pk AS col_is_pk,
      p.dflt_value AS col_default_val,
      p.[notnull] AS col_is_not_null
    FROM sqlite_master m
    LEFT OUTER JOIN pragma_table_info((m.name)) p
      ON m.name <> p.name
    WHERE m.type = 'table'
    ORDER BY table_name, col_id


    """
    cur.execute(sql)
    res= cur.fetchall()
    con.close()
    df=pd.DataFrame(res,columns=["table","col_id","col_name","col_type","col_pk","col_def","col_notnull"])
    return df


df=get_metadata("pack01")
df.head()

,table,col_id,col_name,col_type,col_pk,col_def,col_notnull
0,answer,0,answer_id,INTEGER,1,None,1
1,answer,1,answer_index,SMALLINT,0,None,1
2,answer,2,flag_correct,BOOLEAN,0,None,0
3,question,0,question_id,TEXT,1,None,1
4,question,1,question_text,TEXT,0,None,0


In [4]:
#dump all values from the database
def get_values(path=None,con=None):
    if path:
        print("opening database")
        con=sqlite3.connect(path)
    cur=con.cursor()
    dfdict={}
    df=get_metadata(path)
    for table in df["table"].unique():
        sql=f"""select * from {table}"""
        cur.execute(sql)
        res=cur.fetchall()
        col_list= df[df["table"]==table]["col_name"].to_list()
        dftemp=pd.DataFrame(res,columns=col_list)
        dfdict[table]=dftemp
    con.close()
    return dfdict
dfdict=get_values("pack01")
print(dfdict.keys())
dfdict["test"]

opening database
dict_keys(['answer', 'question', 'test', 'test_category', 'test_question'])


,test_id,title,category_id


In [10]:
def get_schema(path=None,con=None):
    if path:
        con=sqlite3.connect(path)
    cur=con.cursor()
    cur.execute("SELECT type,name,tbl_name,rootpage,sql FROM sqlite_master WHERE type='table';")
    res=cur.fetchall()
    con.close()
    df=pd.DataFrame(res,columns=["type","name","tbl_name","rootpage","sql"])
    df["sql_original"]=df["sql"].copy()
    df["sql"]=df["sql"].replace("\n+"," ",regex=True)
    df["sql"]=df["sql"].replace(" +"," ",regex=True)
    tables_sql=df[["name","sql"]].to_dict(orient="records")
    json_string = json.dumps(tables_sql,indent=4)
    with open("dbspecs.json", "w") as outfile:
        outfile.write(json_string)
    raw_text=";\n".join([sql for sql in df["sql"] ])
    print(raw_text)
    with open("dbspecs.sql", "w") as outfile:
        outfile.write(raw_text)      
    return raw_text
get_schema("pack01")

CREATE TABLE "test_category" ( "category_id" TEXT NOT NULL, PRIMARY KEY ("category_id") );
CREATE TABLE "test" ( "test_id" INTEGER NOT NULL, "title" TEXT NOT NULL, "category_id" TEXT REFERENCES test_category ("category_id") ON UPDATE CASCADE ON DELETE SET NULL, PRIMARY KEY ("test_id") );
CREATE TABLE "test_question" ( "question_id" TEXT NOT NULL REFERENCES question ("question_id") ON UPDATE CASCADE ON DELETE CASCADE, "test_id" INTEGER REFERENCES "test" ("test_id") ON UPDATE CASCADE ON DELETE CASCADE);
CREATE TABLE "question" ( "question_id" TEXT NOT NULL, "question_text" TEXT NULL, "question_notes" TEXT, PRIMARY KEY ("question_id") );
CREATE TABLE "answer" ( "answer_id" INTEGER NOT NULL, "answer_index" SMALLINT NOT NULL, "flag_correct" BOOLEAN NULL, PRIMARY KEY ("answer_id") )


'CREATE TABLE "test_category" ( "category_id" TEXT NOT NULL, PRIMARY KEY ("category_id") );\nCREATE TABLE "test" ( "test_id" INTEGER NOT NULL, "title" TEXT NOT NULL, "category_id" TEXT REFERENCES test_category ("category_id") ON UPDATE CASCADE ON DELETE SET NULL, PRIMARY KEY ("test_id") );\nCREATE TABLE "test_question" ( "question_id" TEXT NOT NULL REFERENCES question ("question_id") ON UPDATE CASCADE ON DELETE CASCADE, "test_id" INTEGER REFERENCES "test" ("test_id") ON UPDATE CASCADE ON DELETE CASCADE);\nCREATE TABLE "question" ( "question_id" TEXT NOT NULL, "question_text" TEXT NULL, "question_notes" TEXT, PRIMARY KEY ("question_id") );\nCREATE TABLE "answer" ( "answer_id" INTEGER NOT NULL, "answer_index" SMALLINT NOT NULL, "flag_correct" BOOLEAN NULL, PRIMARY KEY ("answer_id") )'

In [6]:
def recreate_from_json(json_input_path,sqlite_output_path):
    with open(json_input_path) as f:
        json_string_load = json.load(f)    
    con = sqlite3.connect(":memory:")
    cur= con.cursor()
    for table in json_string_load:
        sql=table["sql"]
        cur.execute(sql)
        con.commit()
    return con
con=recreate_from_json("dbspecs.json",":memory:")
df=get_metadata(con=con)
df.head()

,table,col_id,col_name,col_type,col_pk,col_def,col_notnull
0,answer,0,answer_id,INTEGER,1,None,1
1,answer,1,answer_index,SMALLINT,0,None,1
2,answer,2,flag_correct,BOOLEAN,0,None,0
3,question,0,question_id,TEXT,1,None,1
4,question,1,question_text,TEXT,0,None,0


In [20]:
def recreate_from_text(text_input_path,sqlite_output_path):
    with open(text_input_path) as f:
        sql_file = f.read()    
    con = sqlite3.connect("test_template.db")
    cur= con.cursor()
    for sql in sql_file.split(";"):
        cur.execute(sql)
    con.commit()
    return con
con=recreate_from_text("dbspecs.sql",":memory:")
df=get_metadata(con=con)
df.head(20)

,table,col_id,col_name,col_type,col_pk,col_def,col_notnull
0,answer,0,answer_id,INTEGER,1,None,1
1,answer,1,answer_index,SMALLINT,0,None,1
2,answer,2,flag_correct,BOOLEAN,0,None,0
3,question,0,question_id,INTEGER,1,None,1
4,question,1,question_text,TEXT,0,None,0
5,question,2,question_notes,TEXT,0,None,0
6,test,0,test_id,INTEGER,1,None,1
7,test,1,title,TEXT,0,None,1
8,test,2,category_id,TEXT,0,None,0
9,test_category,0,category_id,INTEGER,1,None,1
